In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('csv files/Car_sales.csv')

df=df[['Manufacturer', 'Sales_in_thousands', 'Vehicle_type', 'Price_in_thousands','Engine_size', 'Horsepower', 'Fuel_capacity']]
df

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
0,Acura,16.919,Passenger,21.50,1.8,140.0,13.2
1,Acura,39.384,Passenger,28.40,3.2,225.0,17.2
2,Acura,14.114,Passenger,NaN,3.2,225.0,17.2
3,Acura,8.588,Passenger,42.00,3.5,210.0,18.0
4,Audi,20.397,Passenger,23.99,1.8,150.0,16.4
...,...,...,...,...,...,...,...
152,Volvo,3.545,Passenger,24.40,1.9,160.0,15.8
153,Volvo,15.245,Passenger,27.50,2.4,168.0,17.9
154,Volvo,17.531,Passenger,28.80,2.4,168.0,17.9
155,Volvo,3.493,Passenger,45.50,2.3,236.0,18.5


In [29]:
df.nunique()

Manufacturer           30
Sales_in_thousands    157
Vehicle_type            2
Price_in_thousands    152
Engine_size            31
Horsepower             66
Fuel_capacity          55
dtype: int64

# 1 agg() 

In [30]:
# calculate sum of rows
df.agg("sum")

Manufacturer          AcuraAcuraAcuraAcuraAudiAudiAudiBMWBMWBMWBuick...
Sales_in_thousands                                             8320.698
Vehicle_type          PassengerPassengerPassengerPassengerPassengerP...
Price_in_thousands                                             4245.567
Engine_size                                                       477.5
Horsepower                                                      29008.0
Fuel_capacity                                                    2800.5
dtype: object

In [31]:
# calculate mean of numeric columns
numeric_cols = df.select_dtypes(include=['number']).columns
df_numcols = df[numeric_cols]
df_numcols.agg("mean")

Sales_in_thousands     52.998076
Price_in_thousands     27.390755
Engine_size             3.060897
Horsepower            185.948718
Fuel_capacity          17.951923
dtype: float64

In [32]:
#calculate mean and count values
df_numcols.apply(['mean', 'count'])

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
mean,52.998076,27.390755,3.060897,185.948718,17.951923
count,157.000000,155.000000,156.000000,156.000000,156.000000


In [33]:
# calculate mean and sum of Sales_in_thousands and the max of Price_in_thousands
df_numcols.agg({'Sales_in_thousands':['mean','sum'], 
                'Price_in_thousands':['max', 'sum']})

,Sales_in_thousands,Price_in_thousands
mean,52.998076,NaN
sum,8320.698000,4245.567
max,NaN,85.500


In [34]:
# aggregate over columns, sum two+ columns
df_numcols[["Sales_in_thousands", "Price_in_thousands"]].agg("sum", axis=1)

0      38.419
1      67.784
2      14.114
3      50.588
4      44.387
        ...  
152    27.945
153    42.745
154    46.331
155    48.993
156    54.969
Length: 157, dtype: float64

In [35]:
# this also works but in row index #2 there is a NaN because it does not treat NaN as a zero like .agg()
df_numcols["Sales_in_thousands"] + df_numcols["Price_in_thousands"]

0      38.419
1      67.784
2         NaN
3      50.588
4      44.387
        ...  
152    27.945
153    42.745
154    46.331
155    48.993
156    54.969
Length: 157, dtype: float64

In [101]:
# agg different functions and rename the index of resulting dataframe
'''
df.agg(
    new_column_name1 = ("existing_column1", "agg_function1"),
    new_column_name2 = ("existing_column2", "agg_function2"),
    ...
)
'''
df[numeric_cols].agg(x =("Sales_in_thousands", "sum"),
                     y =("Price_in_thousands", "sum")) 

,Sales_in_thousands,Price_in_thousands
x,8320.698,NaN
y,NaN,4245.567


# 2 Split-Apply-Combine Strategy

How to best group values in a dataframe to build categories.

In [54]:
# split data into separate groups
passenger_filter = df["Vehicle_type"]=="Passenger"
car_filter = df["Vehicle_type"]=="Car"

# df["Vehicle_type"].value_counts()

In [55]:
# apply an operation
passenger_avg = df[passenger_filter]["Sales_in_thousands"].mean()
car_avg = df[car_filter]["Sales_in_thousands"].mean()

In [57]:
#combine results
pd.DataFrame({"Vehicle_type": ["Car", "Passenger"], 
             "Sales_in_thousands": [car_avg, passenger_avg]})

,Vehicle_type,Sales_in_thousands
0,Car,80.622293
1,Passenger,43.234345


# 3 groupby()

In [61]:
# group by Vehicle_type and calculate the mean
df.groupby("Vehicle_type", as_index=False)[numeric_cols].mean()

,Vehicle_type,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
0,Car,80.622293,26.319975,3.520000,186.400000,21.895000
1,Passenger,43.234345,27.763200,2.902586,185.793103,16.592241


In [67]:
# group by Manufacturer

#get group by object
groupby_obj = df.groupby("Manufacturer")

#show group's attributes and keys (dictionary)
groupby_obj.groups

#get keys only
groupby_obj.groups.keys

<function PrettyDict.keys>

In [70]:
#get a specific group within Manufacturer column
groupby_obj.get_group('Ford')

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
46,Ford,70.227,Passenger,12.070,2.0,110.0,12.7
47,Ford,113.369,Passenger,21.560,3.8,190.0,15.7
48,Ford,35.068,Passenger,17.035,2.5,170.0,15.0
49,Ford,245.815,Passenger,17.885,3.0,155.0,16.0
50,Ford,175.670,Passenger,12.315,2.0,107.0,13.2
51,Ford,63.403,Passenger,22.195,4.6,200.0,19.0
52,Ford,276.747,Car,31.930,4.0,210.0,21.0
53,Ford,155.787,Car,21.410,3.0,150.0,26.0
54,Ford,125.338,Car,36.135,4.6,240.0,26.0
55,Ford,220.650,Car,12.050,2.5,119.0,20.0


In [77]:
# grouby Manufacturer and calculate mean
df.groupby("Manufacturer")[numeric_cols].mean()

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,19.751250,30.633333,2.925000,200.000000,16.400000
Audi,13.519000,39.980000,2.933333,220.000000,19.533333
BMW,15.501667,33.096667,2.700000,185.333333,17.233333
Buick,60.504750,26.781250,3.625000,206.250000,17.750000
Cadillac,22.435600,40.254000,4.500000,256.000000,20.800000


In [79]:
#group by Manufacturer and group by sum
df.groupby("Manufacturer")[numeric_cols].sum()

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,79.005,91.900,11.7,800.0,65.6
Audi,40.557,119.940,8.8,660.0,58.6
BMW,46.505,99.290,8.1,556.0,51.7
Buick,242.019,107.125,14.5,825.0,71.0
Cadillac,112.178,201.270,22.5,1280.0,104.0
Chevrolet,554.365,180.205,27.5,1540.0,139.3
Chrysler,201.721,140.585,16.7,1169.0,98.9
Dodge,910.149,266.350,40.8,2195.0,237.4
Ford,2022.635,231.520,36.6,1871.0,209.7


In [82]:
df.groupby("Vehicle_type")[numeric_cols].count()

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Vehicle_type,,,,,
Car,41,40,40,40,40
Passenger,116,115,116,116,116


In [89]:
# find null values in columns
df.isnull().sum()

Manufacturer          0
Sales_in_thousands    0
Vehicle_type          0
Price_in_thousands    2
Engine_size           1
Horsepower            1
Fuel_capacity         1
dtype: int64

In [93]:
# group by Engine_size and count values without dropping NaN values
# counts Nan values
df.groupby("Engine_size", dropna=False)[numeric_cols].count()

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Engine_size,,,,,
1.0,1,1,1,1,1
1.5,1,1,1,1,1
1.6,1,1,1,1,1
1.8,8,8,8,8,8
1.9,5,5,5,5,5
2.0,17,17,17,17,17
2.2,4,4,4,4,4
2.3,6,6,6,6,6
2.4,11,11,11,11,11


# 4 groupby() and agg() Together

In [98]:
# find the mina nd max values of each column
df.groupby("Vehicle_type")[numeric_cols].min()
df.groupby("Vehicle_type")[numeric_cols].max()

df.groupby("Vehicle_type")[numeric_cols].agg(['min', 'max'])

Sales_in_thousands          Price_in_thousands          \
                            min      max                min     max   
Vehicle_type                                                          
Car                       9.126  540.561             11.528  60.105   
Passenger                 0.110  247.994              9.235  85.500   

             Engine_size      Horsepower        Fuel_capacity        
                     min  max        min    max           min   max  
Vehicle_type                                                         
Car                  2.0  5.7      119.0  300.0          15.1  32.0  
Passenger            1.0  8.0       55.0  450.0          10.3  23.7

In [102]:
# set name for aggregated column
# find min Engine_size and max Horsepower
df.groupby("Vehicle_type")[numeric_cols].agg(min_engine_size=("Engine_size", "min"),
                                             max_horsepower=("Horsepower", "max"))

,min_engine_size,max_horsepower
Vehicle_type,,
Car,2.0,300.0
Passenger,1.0,450.0


In [105]:
# group by Manufacturer calculate the sum of Sales in thousands and mean of Price in thousands
df.groupby("Manufacturer")[numeric_cols].agg(sum_of_sales=("Sales_in_thousands", "sum"),
                                             mean_of_price=("Price_in_thousands", "mean"))

,sum_of_sales,mean_of_price
Manufacturer,,
Acura,79.005,30.633333
Audi,40.557,39.980000
BMW,46.505,33.096667
Buick,242.019,26.781250
Cadillac,112.178,40.254000
Chevrolet,554.365,20.022778
Chrysler,201.721,23.430833
Dodge,910.149,24.213636
Ford,2022.635,21.047273


# 5 Lambda on groupby()

In [108]:
df.groupby("Manufacturer").sum()

,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,,
Acura,79.005,PassengerPassengerPassengerPassenger,91.900,11.7,800.0,65.6
Audi,40.557,PassengerPassengerPassenger,119.940,8.8,660.0,58.6
BMW,46.505,PassengerPassengerPassenger,99.290,8.1,556.0,51.7
Buick,242.019,PassengerPassengerPassengerPassenger,107.125,14.5,825.0,71.0
Cadillac,112.178,PassengerPassengerPassengerPassengerCar,201.270,22.5,1280.0,104.0
Chevrolet,554.365,PassengerPassengerPassengerPassengerPassengerP...,180.205,27.5,1540.0,139.3
Chrysler,201.721,PassengerPassengerPassengerPassengerPassengerC...,140.585,16.7,1169.0,98.9
Dodge,910.149,PassengerPassengerPassengerPassengerPassengerC...,266.350,40.8,2195.0,237.4
Ford,2022.635,PassengerPassengerPassengerPassengerPassengerP...,231.520,36.6,1871.0,209.7


In [113]:
# get actual sales and price values
df.groupby("Manufacturer")[numeric_cols].sum().apply(lambda x:x*1000)[["Sales_in_thousands","Price_in_thousands"]]

,Sales_in_thousands,Price_in_thousands
Manufacturer,,
Acura,79005.0,91900.0
Audi,40557.0,119940.0
BMW,46505.0,99290.0
Buick,242019.0,107125.0
Cadillac,112178.0,201270.0
Chevrolet,554365.0,180205.0
Chrysler,201721.0,140585.0
Dodge,910149.0,266350.0
Ford,2022635.0,231520.0


In [132]:
# subtract values from the mean of each 

#this is subtracting the value in each numeric column by the mean/avg of each column
'''
In a nutshell:
df.groupby("Manufacturer")[numeric_cols] - df.groupby("Manufacturer")[numeric_cols].mean()
'''
df.groupby("Manufacturer")[numeric_cols].apply(lambda x: x - x.mean())

Sales_in_thousands  Price_in_thousands  Engine_size  \
Manufacturer                                                            
Acura        0             -2.832250           -9.133333    -1.125000   
             1             19.632750           -2.233333     0.275000   
             2             -5.637250                 NaN     0.275000   
             3            -11.163250           11.366667     0.575000   
Audi         4              6.878000          -15.990000    -1.133333   
...                              ...                 ...          ...   
Volvo        152           -9.078333           -6.533333    -0.400000   
             153            2.621667           -3.433333     0.100000   
             154            4.907667           -2.133333     0.100000   
             155           -9.130333           14.566667     0.000000   
             156            6.345667            5.066667     0.600000   

                  Horsepower  Fuel_capacity  
Manufacturer                                 
Acura        0    -60.000000      -3.200000  
             1     25.000000       0.800000  
             2     25.000000       0.800000  
             3     10.000000       1.600000  
Audi         4    -70.000000      -3.133333  
...                      ...            ...  
Volvo        152  -22.166667      -2.033333  
             153  -14.166667       0.066667  
             154  -14.166667       0.066667  
             155   53.833333       0.666667  
             156   18.833333       3.266667  

[157 rows x 5 columns]

In [129]:
df

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
0,Acura,16.919,Passenger,21.50,1.8,140.0,13.2
1,Acura,39.384,Passenger,28.40,3.2,225.0,17.2
2,Acura,14.114,Passenger,NaN,3.2,225.0,17.2
3,Acura,8.588,Passenger,42.00,3.5,210.0,18.0
4,Audi,20.397,Passenger,23.99,1.8,150.0,16.4
...,...,...,...,...,...,...,...
152,Volvo,3.545,Passenger,24.40,1.9,160.0,15.8
153,Volvo,15.245,Passenger,27.50,2.4,168.0,17.9
154,Volvo,17.531,Passenger,28.80,2.4,168.0,17.9
155,Volvo,3.493,Passenger,45.50,2.3,236.0,18.5


In [133]:
df.groupby("Manufacturer")[numeric_cols].mean()

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,19.751250,30.633333,2.925000,200.000000,16.400000
Audi,13.519000,39.980000,2.933333,220.000000,19.533333
BMW,15.501667,33.096667,2.700000,185.333333,17.233333
Buick,60.504750,26.781250,3.625000,206.250000,17.750000
Cadillac,22.435600,40.254000,4.500000,256.000000,20.800000
Chevrolet,61.596111,20.022778,3.055556,171.111111,15.477778
Chrysler,28.817286,23.430833,2.783333,194.833333,16.483333
Dodge,82.740818,24.213636,3.709091,199.545455,21.581818
Ford,183.875909,21.047273,3.327273,170.090909,19.063636


# 6 Filter values by aggregrate function

In [153]:
#filter certain values out based on sum() and return a subset

# manufacturers whose Sales_in_thousands are greater than the avg across all manufacturers
def filter_func(x):
    return x["Sales_in_thousands"].sum() > df["Sales_in_thousands"].mean()

df_filter = df.groupby(["Manufacturer"]).filter(filter_func)
df_filter

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
0,Acura,16.919,Passenger,21.500,1.8,140.0,13.2
1,Acura,39.384,Passenger,28.400,3.2,225.0,17.2
2,Acura,14.114,Passenger,NaN,3.2,225.0,17.2
3,Acura,8.588,Passenger,42.000,3.5,210.0,18.0
10,Buick,91.561,Passenger,21.975,3.1,175.0,17.5
...,...,...,...,...,...,...,...
152,Volvo,3.545,Passenger,24.400,1.9,160.0,15.8
153,Volvo,15.245,Passenger,27.500,2.4,168.0,17.9
154,Volvo,17.531,Passenger,28.800,2.4,168.0,17.9
155,Volvo,3.493,Passenger,45.500,2.3,236.0,18.5


In [158]:
# calculate which Manufacturers were filtered out
df.groupby(["Manufacturer"])[numeric_cols].sum()["Sales_in_thousands"].sort_values()

# first 6 sums' were less than mean (~52.99)
df[df["Manufacturer"].isin(["BMW", "Audi", "Inifniti", "Saab", "Jaguar", "Porsche"])].shape

(12, 7)

In [157]:
# lost 13 rows that belonged to those 6 manufacturers because they didnt pass the filter_func
print(df.shape)
print(df_filter.shape)

(157, 7)
(144, 7)
